# Importación de los países en la base de datos

### 1. Importar librerías y cargar datos

En esta sección importamos las librerías necesarias (`pandas`, `os`, `dotenv` y `pymysql`), cargamos las variables de entorno desde un archivo `.env` y leemos el CSV con la información de países, subcontinentes y continentes.


In [7]:
import os
from dotenv import load_dotenv
import pandas as pd
import pymysql
from pymysql.constants import CLIENT
 
load_dotenv()
DB_HOST     = os.getenv('DB_HOST')
DB_USER     = os.getenv('DB_USER')
DB_PASSWORD = os.getenv('DB_PASSWORD')
DB_NAME     = os.getenv('DB_NAME')

df_geo = pd.read_csv('../../data/fuentes/paises/Pa_sos.csv')
df_geo.head()


,Codi,Nom,Codi A3,Codi Divisa,Codi Regió UN M49,Continent Codi,Continent Nom,Subcontinent Codi,Subcontinent Nom,Nom en anglès,Nom en castellà
0,AF,Afganistan,AFG,AFN,34,4,Àsia,12,Àsia meridional,Afghanistan,Afganistán
1,AX,Aland,ALA,EUR,154,5,Europa,17,Unió Europea,Aland Islands,Islas Aland
2,AL,Albània,ALB,ALL,39,5,Europa,16,Europa (no UE),Albania,Albania
3,DE,Alemanya,DEU,EUR,155,5,Europa,17,Unió Europea,Germany,Alemania
4,DZ,Algèria,DZA,DZD,15,1,Àfrica,5,Àfrica septentrional,Algeria,Argelia


### 2. Traducción de nombres

Los campos `Continent Nom` y `Subcontinent Nom` están en catalán. Definimos diccionarios de traducción y creamos nuevas columnas en castellano.


In [8]:
continent_map = {
    'Àsia':'Asia',
    'Europa':'Europa',
    'Àfrica':'África',
    'Amèrica':'América',
    'Antàrtida i territoris propers':'Antártida y territorios cercanos',
    'Oceania':'Oceanía'
}

# Aplicar traducción
df_geo['continent_es']= df_geo['Continent Nom'].map(continent_map)

# Verificar
df_geo[['Continent Nom','continent_es']].drop_duplicates().head()


,Continent Nom,continent_es
0,Àsia,Asia
1,Europa,Europa
4,Àfrica,África
7,Amèrica,América
8,Antàrtida i territoris propers,Antártida y territorios cercanos


### 3. Conexión a la base de datos

Abrimos una conexión a MySQL usando `pymysql`, con la bandera `MULTI_STATEMENTS` para permitir múltiples sentencias.


In [9]:
# 1) Conexión
conexion = pymysql.connect(
    host=DB_HOST,
    user=DB_USER,
    password=DB_PASSWORD,
    database=DB_NAME,
    client_flag=CLIENT.MULTI_STATEMENTS
)
cursor = conexion.cursor()

### 4. Inserción de Continentes

Extraemos los continentes únicos, asignamos su `id` (tal como vienen en el CSV) y los insertamos.

In [10]:
# 1) Extraer y desduplicar
continentes = df_geo[['Continent Codi', 'continent_es']].drop_duplicates() 
# 2) Filtrar filas con código o nombre nulo
continentes = continentes.dropna(subset=['Continent Codi', 'continent_es']) 
# 3) Convertir el código a entero
continentes['Continent Codi'] = continentes['Continent Codi'].astype(int) 
# 4) Ahora sí, preparar los datos y ejecutar el INSERT
sql_ins_cont = "INSERT INTO Continentes (id, nombre) VALUES (%s, %s)" 
print(continentes) 
data = list(zip( continentes['Continent Codi'], continentes['continent_es'] )) 
cursor.executemany(sql_ins_cont, data) 
conexion.commit() 
print("Continentes insertados:", len(data))

    Continent Codi                      continent_es
0                4                              Asia
1                5                            Europa
4                1                            África
7                2                           América
8                3  Antártida y territorios cercanos
14               6                           Oceanía
Continentes insertados: 6


### 5. Inserción de Países

Por último, insertamos cada país con su código, nombre en castellano y la referencia al subcontinente.

In [11]:
paises = df_geo[['Codi','Nom en castellà', 'Nom en anglès','Continent Codi']].drop_duplicates()
paises = paises.dropna(subset=['Codi','Nom en castellà','Nom en anglès','Continent Codi'])
paises['Codi'] = paises['Codi'].astype(str) 
paises['Continent Codi'] = paises['Continent Codi'].astype(int)
paises['Nom en anglès'] = paises['Nom en anglès'] .str.replace(r'\s*\(the\)$', '', regex=True) 
# 🔹 Filtrar países no deseados
paises = paises[~paises['Nom en castellà'].isin(['Otros/Varios', 'No consta'])]
data = list(zip( paises['Codi'], paises['Nom en castellà'], paises['Nom en anglès'], paises['Continent Codi'] ))
sql_ins_pais = """ INSERT INTO Paises (codigo, nombre, nombre_en, continente_id) VALUES (%s, %s, %s, %s) """
cursor.executemany(sql_ins_pais, data)
conexion.commit()
print("Países insertados:", len(data))

Países insertados: 248


### 7. Cierre de la conexión

Cerramos el cursor y la conexión a la base de datos.

In [12]:
cursor.close()
conexion.close()
print("Conexión cerrada.")

Conexión cerrada.
